# Ask & Tell Interface

## 1. 최소한의 수정 문제에 Optuna 적용하기 

In [5]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import optuna

X, y = make_classification(n_features = 10)
X_train, X_test, y_train, y_test = train_test_split(X, y)

C = 0.01
clf = LogisticRegression(C = C)
clf.fit(X_train, y_train)
val_accuracy = clf.score(X_test, y_test)

- 이후 하이퍼파라미터를 C와 solver를 이용해 최적화하고 싶다면~

In [7]:
def objective(trial):
    X, y = make_classification(n_features = 10)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    C = trial.suggest_float("C", 1e-7, 10.0, log = True)
    solver = trial.suggest_categorical('solver', ('lbfgs', 'saga'))
    
    clf = LogisticRegression(C = C, solver = solver)
    clf.fit(X_train, y_train)
    val_accuracy = clf.score(X_test, y_test)
    
    return val_accuracy

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 10)

[I 2022-12-12 15:54:23,619] A new study created in memory with name: no-name-85936349-b0a8-40c9-9b01-f3f1b9a8f74e
[I 2022-12-12 15:54:23,636] Trial 0 finished with value: 0.92 and parameters: {'C': 0.003038349854617212, 'solver': 'saga'}. Best is trial 0 with value: 0.92.
[I 2022-12-12 15:54:23,651] Trial 1 finished with value: 0.48 and parameters: {'C': 8.08989303264751e-05, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.92.
[I 2022-12-12 15:54:23,661] Trial 2 finished with value: 0.92 and parameters: {'C': 0.1745187608259836, 'solver': 'saga'}. Best is trial 0 with value: 0.92.
C:\Users\dowra\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-12-12 15:54:23,676] Trial 3 finished with value: 0.36 and parameters: {'C': 1.2248026201238313e-05, 'solver': 'saga'}. Best is trial 0 with value: 0.92.
[I 2022-12-12 15:54:23,690] Trial 4 finished with value: 0.88 and param

- 위 결과를 보면 요런 말이 있음
```
ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
```
- `trial` 외에 `objective`가 추가적인 인수가 필요하다면 아예 class를 정의해야 함
- 아래 예제는 위와 동일하다

In [10]:
study = optuna.create_study(direction = 'maximize')

n_trials = 10

for _ in range(n_trials):
    trial = study.ask() # trial은 Trial임. FrozenTrial이 아님!
    
    C = trial.suggest_float("C", 1e-7, 10.0, log = True)
    solver = trial.suggest_categorical('solver', ('lbfgs', 'saga'))
    
    clf = LogisticRegression(C=C, solver = solver)
    clf.fit(X_train, y_train)
    val_accuracy = clf.score(X_test, y_test)
    
    # tell 메서드를 이용해 trial과 objective_value를 같이 준다
    study.tell(trial, val_accuracy)

[I 2022-12-12 15:58:49,670] A new study created in memory with name: no-name-39e9e81d-6a10-4093-a86e-b5f17e7dfcc2


- 다른점은 2가지임
1. `optuna.study.Study.ask()` : 하이퍼파라미터를 샘플링하는 trial을 만듦
2. `optuna.study.Study.tell()` : trial을 완료함. 

- 위 방법을 이용하면 `objective function` **없이 하이퍼파라미터 최적화를 이용할 수 있음**

In [14]:
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier

X, y = load_iris(return_X_y = True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
classes = np.unique(y)
n_train_iter = 100

study = optuna.create_study(
    direction = 'maximize',
    pruner = optuna.pruners.HyperbandPruner(
        min_resource = 1, max_resource = n_train_iter, reduction_factor = 3
    )
)

for _ in range(20):
    trial = study.ask()
    
    alpha = trial.suggest_float('alpha', 0.0, 1.0)
    
    clf = SGDClassifier(alpha = alpha)
    pruned_trial = False
    
    for step in range(n_train_iter):
        clf.partial_fit(X_train, y_train, classes = classes)
        
        intermediate_value = clf.score(X_valid, y_valid)
        trial.report(intermediate_value, step)
        
        if trial.should_prune():
            pruned_trial = True
            break
            
    if pruned_trial:
        study.tell(trial, state = optuna.trial.TrialState.PRUNED)
    else:
        score = clf.score(X_valid, y_valid)
        study.tell(trial, score)

[I 2022-12-12 16:05:06,950] A new study created in memory with name: no-name-03f10ca1-7e4a-4879-b6ee-9b9467a1df03


- `optuna.study.Study.tell()` 메서드는 `trial` 객체보다 더 많은 수를 가질 수 있다. 
- `study.tell(trial.number, y)` = `study.tell(trial, y)`

## Define and Run

- 하이퍼파라미터의 분포를 `ask()`를 호출하기 전에 정의할 수 있음

In [15]:
distributions = {
    'C' : optuna.distributions.FloatDistribution(1e-7, 10.0, log = True),
    'solver' : optuna.distributions.CategoricalDistribution(('lbfgs', 'saga')),
}

`distributions`을  `optuna.study.Study.ask()`에 통과시켜보자
- 위에서는 `study.ask()` 를 정의한 다음 `suggest`를 이용했다면, 여기서는 하이퍼파라미터의 범위를 `dict`로 정의한 다음 `ask()` 내에 통과시켜줬음

In [19]:
X_train.shape

(25, 10)

In [20]:
study = optuna.create_study(direction = 'maximize')
n_trials = 10

for _ in range(n_trials):
    trial = study.ask(distributions)
    
    # 사전에 분포를 정의했기 떄문에 `ask()`를 통해 안에 파라미터로 들어가 있음
    C = trial.params['C']
    solver = trial.params['solver']
    
    clf = LogisticRegression(C = C, solver = solver)
    clf.fit(X_train, y_train)
    val_accuracy = clf.score(X_test, y_test)
    
    study.tell(trial, val_accuracy)

[I 2022-12-12 16:13:32,741] A new study created in memory with name: no-name-35c1455c-6ab5-4c60-acba-57c6d665037b
C:\Users\dowra\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dowra\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dowra\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dowra\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dowra\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.wa

## Batch Optimization

- Ask and Tell 인터페이스를 이용하면 일괄 처리된 목표를 더 빠르게 최적화할 수 있다
- 병렬화 가능한 평가, 벡터 작업 등


In [25]:
def batched_objective(xs: np.ndarray, ys: np.ndarray):
    return xs ** 2 + ys

- 아래 예제에서 batch의 하이퍼파라미터 쌍 갯수는 10개이고, `batched_objective`는 3번 평가된다. 따라서 총 trial 수는 30개이다.
- batch 평가 후 `trial_numbers`나 `trial`을 저장하려면 `optuna.study.Study.tell()`메서드를 호출해야 한다.

In [26]:
batch_size = 10
study = optuna.create_study(sampler = optuna.samplers.CmaEsSampler())

for _ in range(3): # 각각이 1개의 batch
    trial_numbers = []
    x_batch = []
    y_batch = []
    
    for _ in range(batch_size):

        trial = study.ask()
        trial_numbers.append(trial.number)
        x_batch.append(trial.suggest_float('x', -10, 10))
        y_batch.append(trial.suggest_float('y', -10, 10))
        
    x_batch = np.array(x_batch)
    y_batch = np.array(y_batch)
    objectives = batched_objective(x_batch, y_batch)
    
    for trial_number, objective in zip(trial_numbers, objectives):
        study.tell(trial_number, objective)

[I 2022-12-12 16:20:27,950] A new study created in memory with name: no-name-690b1afd-b3fa-4f62-a62d-14ddf8712e17
